In [ ]:
# для транскрибации тинькоф
!pip install tinkoff-voicekit-client protobuf==3.20.3

In [ ]:
# для транскрибации виспером
!pip install git+https://github.com/openai/whisper.git

In [ ]:
# только для транскрибации
import whisper
import time

In [1]:
# для анализатора текста диалога
!pip install -q nest_asyncio xmltodict faiss-cpu==1.7.4 langchain==0.1.7 openai==1.12.0 tiktoken==0.6.0 langchain_community==0.0.20 langchain-openai==0.0.6 unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 58.5 MB/s eta 0:00:00
  Preparing metadata (se

In [2]:
# подключение гугл диска
import os
from google.colab import drive
mountpoint = '/content/drive'
drive.mount(mountpoint, force_remount=True)
initial_directory = os.getcwd()
print(initial_directory)
# установить текущую папку для долговременного сохранения файлов
folder_path  = '/content/drive/MyDrive/docs4colab/data_structure'
# сделать папку текущей, чтобы не указывать полный путь для файловых операций
os.chdir(folder_path)
current_directory = os.getcwd()
print(current_directory)

Mounted at /content/drive
/content
/content/drive/MyDrive/docs4colab/data_structure


In [3]:
# для анализа диалога
# fixes a bug with asyncio and jupyter
import nest_asyncio
nest_asyncio.apply()
from IPython.display import display, HTML
from IPython.display import display, HTML, clear_output
from IPython.display import display_html
import openai
from openai import OpenAI
import subprocess
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import requests
import re
import tiktoken
import os
import json
from datetime import datetime

from google.colab import userdata

# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key


In [5]:
def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    """Возвращает количество токенов в строке"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
# определение стоимости запроса в рублях при курсе 90руб/1$
def cost(dictusage,model):
    # Model Input Output
    # gpt-4o 5,00 $ / 1M tokens 15,00 $ / 1M tokens
    # gpt-4o-2024-05-13 5,00 $ /  1M tokens 15,00 $ / 1M tokens
    # gpt-3.5-turbo-0125 0,50 $ / 1M tokens 1,50 $ / 1M tokens

    tarif={"gpt-4o":[5/1000000,15/1000000],
           "gpt-4o-2024-05-13":[5/1000000,15/1000000],
           "gpt-3.5-turbo-0125" :[0.50/1000000,1.50/1000000],
           }
    cursrubl=90
    # CompletionUsage(completion_tokens=4065, prompt_tokens=4573, total_tokens=8638)
    cost=cursrubl*tarif[model][0]*dictusage.__dict__['prompt_tokens']+cursrubl*tarif[model][1]*dictusage.__dict__['completion_tokens']
    return "{:.4f}".format(cost)


# Системный промпт

In [46]:
system="""

Ты опытный сотрудник отдела маркетинга и сервиса в компании, которая продает электронику и предоставляет услуги по ремонту и консультации.
Ты специалист по конструктивному ведению диалогов, обладающий высокой компетентностью в вопросах характеристик, особенностей использования и ремонта различных гаджетов.
Ты также умеешь отлично анализировать прошедшие диалоги менеджеров с клиентами, выделять основные тренды, моменты их изменения.

Текст диалога между менеджером и клиентом будет предоставлен тебе в формате JSON как список словарей
с репликами менеджера по ключу "1", репликами клиента по ключу "2".

Тебе будет предоставлена в формате JSON блок-схема, описывающая все допустимые сценарии диалога менеджера с клиентом. Схема изначально не содержит текста диалога.
Диалог возможен в свободной форме, но содержание и последовательность его фрагментов строго регламентированы.
Блок-схема описывает диалог в виде серии вложенных и последовательных секций и задает разделение на альтернативные ветки по условию и слияние разных веток в случае, если их продолжение идентично.
Секция представлена в формате ключ/словарь;секция с ключем "root" является точкой входа в диалог.
в словаре секции пары ключ/значение описывают допустимое содержание фрагмента диалога и условия ветвлений и слияний.
Фрагмент диалога в данном контексте представляет содержательную часть, в которой менеджер решает определенную подзадачу, которая приближает к достижению главной цели всего диалога.
В общем случае секция может являться только инструкцией перехода и содержать только ключ слияния.
пример:
сценарий диалога развивается от секции root, далее переходит к вложенной секции consultation по ее условию description .
альтернативный сценарий развивается от секции root, далее переходит к вложенной секции availability по ее условию description.
и т.д.

Описание пар ключ/значение для словаря секции:

description: текст - описание ключевого события или условия, определяющего переход к данной секции от предыдущей и означающее начало новой подзадачи для менеджера.

task: текст - основной план или подзадача, которых должен придерживаться менеджер в данном фрагменте диалога для достижения конечной цели всего диалога,
если текст по ключу task пустой, то выполняемая задача не требует обязательного диалога с клиентом.Если task не заполнен, то для содержания секции существенно только условие по ключу description.

steps: список словарей с опциональным одним из ключей agent_action или client_action, предназначенных для реплик менеджера и клиента соответственно;
этот список предназначен для заполнения ChatGPT и изначально пуст.

branching: список ключей, отражающих допустимые варианты ветвления по условию в данной точке диалога; управление переходит только по одному из этих ключей к следующей вложенной секции,
у которой ключ description наиболее близко соответствует условию перехода.

merge: ключ слияния - инструкция перехода, указывающий на секцию, являющуюся одинаковым продолжением нескольких альтернативных веток.
Ключ слияния следует искать на одном уровне с ключом root.

В общем случае развитие диалога конструируется из последовательности секций,
где выбор каждой следующей секции строго ограничен списком по ключу branching или значением по ключу merge, определенными в текущей секции.

1. Сконструируй по блок-схеме все возможные сценарии течения диалога в форме последовательности секций.
2. Для каждого варианта из пункта 1 сформируй отдельный текст-конспект как конкатенацию значений по ключу description и значений по ключу task и создий список результатов.
пример для двух из всех возможных сценариев:
"root": { "description": "Входящий звонок клиента. Начало диалога.",
      "task": "Выявление первичной потребности клиента",
      "branching": ["availability","consultation",  "help", "store_info", "reservation", "repair", "other"],
      "consultation": {
        "description": "Интересует консультация по товару",
        "task": "Осуществляем краткосрочную консультацию по товару",}
      "availability": {
        "description": "Вопрос клиента по наличию товара",
        "task": "Бла-бла-бла.",},
        и т.д.}
ответ к примеру:
result:["Входящий звонок клиента. Начало диалога.Выявление первичной потребности клиента.Интересует консультация по товару.Осуществляем краткосрочную консультацию по товару",
"Входящий звонок клиента. Начало диалога.Выявление первичной потребности клиента.Вопрос клиента по наличию товара.Бла-бла-бла."
]
3. Используй полученные тексты-конспекты как образцы результатов суммаризации диалогов менеджера и клиента. Суммаризируй переданный тебе диалог менеджера и клиента по образцам.

**Представление результатов в виде JSON на основе предоставленной тебе блок-схемы:**
верни результат из пункта 3
"""

In [62]:
system="""
Ты опытный сотрудник отдела маркетинга и сервиса в компании, которая продает электронику и предоставляет услуги по ремонту и консультации.
Ты специалист по конструктивному ведению диалогов, обладающий высокой компетентностью в вопросах характеристик, особенностей использования и ремонта различных гаджетов.
Ты также умеешь отлично анализировать прошедшие диалоги менеджеров с клиентами, выделять основные тренды, моменты их изменения.
Главной целью менеджера в диалоге является привлечение клиента к использованию услуг компании и/или покупке товаров компании.

Тебе будет предоставлен текст диалога между менеджером и клиентом в формате JSON, как список словарей с репликами менеджера по ключу "1" и репликами клиента по ключу "2".

Тебе также будет предоставлена блок-схема в формате JSON, описывающая все допустимые сценарии диалога менеджера с клиентом. Схема изначально не содержит текста диалога.
Диалог возможен в свободной форме, но содержание и последовательность его фрагментов строго регламентированы.
Блок-схема описывает диалог в виде серии вложенных и последовательных секций и задает разделение на альтернативные ветки по условию и слияние разных веток в случае, если их продолжение идентично.
Секция представлена в формате ключ/словарь; секция с ключем "root" является точкой входа в диалог.
В словаре секции пары ключ/значение описывают условие перехода к данной секции,допустимое содержание фрагмента диалога и варианты дальнейших ветвлений; в общем случае оканчиваются ключем слияние.
Фрагмент диалога в данном контексте представляет содержательную часть, в которой менеджер решает определенную подзадачу, которая приближает к достижению главной цели всего диалога.
В общем случае секция может являться только инструкцией перехода и содержать только ключ слияния.

Основные инструкции
1.Сконструировать сценарии диалога:

Сконструируй все возможные сценарии диалога согласно блок-схеме в форме последовательности секций.
2.Создать резюме:

Для каждого сценария из шага 1 создай отдельное текстовое резюме путем объединения значений по ключам description и task. Составь список этих резюме.
Пример:
json
{
  "result": [
    "Входящий звонок клиента. Начало диалога. Выявление первичной потребности клиента. Интересует консультация по товару. Осуществляем краткосрочную консультацию по товару.",
    "Входящий звонок клиента. Начало диалога. Выявление первичной потребности клиента. Вопрос клиента по наличию товара. Предоставляем информацию о наличии товара."
  ]
}
3.Суммировать диалог:

Используй созданные резюме в качестве шаблонов для суммирования предоставленного диалога между менеджером и клиентом.
Финальное резюме должно отражать общую структуру и содержание диалога, следуя образцам, установленным созданными резюме.
4.Найти сценарий:
Из всего списка по пункту 2 найди резюме, которое более всех остальных совпадает с суммаризованным диалогом менеджера и клиента. Используй эмбеддинги для определения степени совпадения.
Найди сценарий из списка 1, который соответствует этому резюме по порядковому номеру.
5.Раздели исходный диалог менеджера и клиента:
Раздели диалог на фрагменты и сопоставь фрагменты с секциями сценария из пункта 4 так, чтобы
интегральное посекционное соответствие фрагментов диалога планам (task) и условиям (description) было максимально возможным.
Обязательно используй все секции сценария.Ничего не пропускай.Из секции в секцию переход возможен только согласно блок-схеме по условиям ветвления и безусловным слияниям.
Используй эмбеддинги для определения степени совпадения.
JSON Вывод
Верни результат из шага 5 в формате JSON как список словарей с ключами секций и значениями - фрагментами диалога."""

In [42]:
client = OpenAI()
# загрузить формализованный скрипт диалога и текст диалога
def load_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# загрузить формализованную схему скрипта
shem= load_json_from_file('dialstructure6.json')
# загрузить json диалог, разделенный по каналам на менеджера и клиента
dial= load_json_from_file("2024-05-24_15-28-08.674147_from_79171173909_to_033222_session_4012586103_talk.json")

In [43]:
print(str(shem))
print(str(dial))

{'root': {'description': 'Входящий звонок клиента. Начало диалога.', 'task': 'Выявление первичной потребности клиента', 'steps': [{'agent_action': 'Добрый день, меня зовут ......, какой у Вас вопрос?', 'client_action': 'Скажите, пожалуйста,......'}], 'branching': ['availability', 'consultation', 'help', 'store_info', 'reservation', 'repair', 'other'], 'consultation': {'description': 'Клиента интересует консультация по товару', 'task': 'Осуществляем краткосрочную консультацию по товару', 'steps': [{'agent_action': '', 'client_action': ''}], 'branching': ['instock', 'emptystock', 'toorder'], 'instock': {'description': 'Товар есть в наличии', 'task': '', 'steps': [{'agent_action': '', 'client_action': ''}], 'merge': 'common_order_process'}, 'emptystock': {'description': 'Товар кончился на складе, уже заказывали, знаем условия заказа', 'task': 'Ориентируем клиента по срокам и конечной стоимости', 'steps': [{'agent_action': '', 'client_action': ''}], 'merge': 'common_order_process'}, 'toord

# Анализ диалога.

In [15]:
porogmax=16000
def answer_index(system,shem,dial,model="gpt-4o",verbose=0):
    global porogmax
    global client

    # Начало операции
    start_time = datetime.now()
    porog=porogmax
    messages = [
        {"role": "system", "content": system,}
    ]
    messages.append({"role": "user", "content": f"JSON блок-схема диалога:\n{str(shem)}\nJSON текст диалога менеджера и клиента:\n{str(dial)}"})
    for mess in messages:
        porog-=num_tokens_from_string(mess['content'])

    print(f"Первичная оценка размера запроса в токенах: {porogmax-porog}")
    if porog<0: return -1,f"Размер запроса {porogmax-porog} превышает допустимый порог porogmax",-1

    completion = client.chat.completions.create(
        model=model, #"gpt-3.5-turbo-0125", #gpt-3.5-turbo-1106
        response_format={ "type": "json_object" },
        messages=messages,
        temperature=0
    )
    dictusage=completion.__dict__['usage']
    json_answer = completion.choices[0].message.content
    if verbose:
        # Конец операции
        end_time = datetime.now()
        # Вычисляем продолжительность операции
        duration = end_time - start_time
        # Преобразуем продолжительность в минуты и секунды
        minutes, seconds = divmod(duration.total_seconds(), 60)
        current_time = end_time.strftime("%Y-%m-%d %H:%M")
        print(f"Анализ диалога закончен в {current_time}; продолжительность анализа {int(minutes)} мин {int(seconds)} сек; ", dictusage)
        print(f"Полная стоимость анализа диалога: {dictusage.__dict__['total_tokens']} токенов, {cost(dictusage,model)} рублей")
        print(f'{"json_answer":=^50}\n', json_answer)
    try:
        # загрузить json объект из ответа
        # Попытка преобразовать извлеченный JSON в объект Python
        json_obj = json.loads(json_answer)
        # Если преобразование прошло успешно, вы можете продолжить работу с json_obj
    except json.JSONDecodeError as e:
        # Обработка исключения, возникающего при некорректном JSON
        return -1,f"<div>повторите еще раз этот же вопрос</br>т.к. из-за сбоя модели анализатор вернул некорректную структуру:</br>{json_answer}</div>",completion.__dict__['usage']

    return 0,json_obj,completion.__dict__['usage']

In [59]:
model="gpt-3.5-turbo-0125"
result,answer,dictusage=answer_index(system,shem,dial,model,verbose=1)
if result<0:
    print(f"ошибка {str(answer)}")
else:
    print(f"Полная стоимость анализа диалога: {dictusage.__dict__['total_tokens']} токенов, {cost(dictusage,model)} рублей")
    print(f"{str(answer)}")

Первичная оценка размера запроса в токенах: 5386
Анализ диалога закончен в 2024-06-18 13:45; продолжительность анализа 0 мин 7 сек;  CompletionUsage(completion_tokens=293, prompt_tokens=5397, total_tokens=5690)
Полная стоимость анализа диалога: 5690 токенов, 0.2824 рублей
===================json_answer====================
 {
  "result": [
    "Входящий звонок клиента. Начало диалога. Выявление первичной потребности клиента. Вопрос клиента по наличию товара. Предоставление актуальной информации по товару и где он находится. Клиент интересуется наличием iPhone Pro Max 15. Уточнение цвета и объема памяти. Предложение актуальной стоимости и способов оплаты.",
    "Входящий звонок клиента. Начало диалога. Выявление первичной потребности клиента. Вопрос клиента по наличию товара. Предоставление актуальной информации по товару и где он находится. Клиент интересуется наличием iPhone Pro Max 15. Уточнение цвета и объема памяти. Предложение актуальной стоимости и способов оплаты. Обсуждение возм

In [63]:
model="gpt-4o"
result,answer,dictusage=answer_index(system,shem,dial,model,verbose=1)
if result<0:
    print(f"ошибка {str(answer)}")
else:
    print(f"Полная стоимость анализа диалога: {dictusage.__dict__['total_tokens']} токенов, {cost(dictusage,model)} рублей")
    print(f"{str(answer)}")

Первичная оценка размера запроса в токенах: 5548
Анализ диалога закончен в 2024-06-18 13:54; продолжительность анализа 1 мин 27 сек;  CompletionUsage(completion_tokens=2154, prompt_tokens=3891, total_tokens=6045)
Полная стоимость анализа диалога: 6045 токенов, 4.6589 рублей
===================json_answer====================
 
{
  "result": [
    {
      "root": {
        "description": "Входящий звонок клиента. Начало диалога.",
        "task": "Выявление первичной потребности клиента",
        "dialog": [
          {
            "1": "Добрый день, магазин IT. Меня зовут Игнат. Чем могу помочь?",
            "2": "Здравствуйте, меня зовут Артем. Я хотел у вас спросить, а у вас есть в наличии iPhone Pro Max 15?"
          }
        ]
      }
    },
    {
      "availability": {
        "description": "Вопрос клиента по наличию товара",
        "task": "",
        "dialog": [
          {
            "1": "Да, уточните, пожалуйста, какой цвет и объем памяти вас интересует?",
            "

In [38]:
model="gpt-4o-2024-05-13"
result,answer,dictusage=answer_index(system,shem,dial,model,verbose=1)
if result<0:
    print(f"ошибка {str(answer)}")
else:
    print(f"Полная стоимость анализа: {dictusage.__dict__['total_tokens']} токенов, {cost(dictusage,model)} рублей")
    print(f"{str(answer)}")

Первичная оценка размера запроса в токенах: 7478
Анализ диалога закончен в 2024-06-17 11:43; продолжительность анализа 0 мин 49 сек;  CompletionUsage(completion_tokens=3122, prompt_tokens=5159, total_tokens=8281)
Полная стоимость анализа диалога: 8281 токенов, 0.0726 рублей
===================json_answer====================
 {
  "call_center_workflow": {
    "info": "info0,1,2,...,n - пояснения ключей",
    "root": {
      "description": "Входящий звонок клиента. Начало диалога.",
      "task": "Выявление первичной потребности клиента",
      "steps": [
        {
          "agent_action": "Добрый день, магазин IT. Меня зовут Игнат. Чем могу помочь?",
          "client_action": "Здравствуйте, меня зовут Артем. Я хотел у вас спросить, а у вас есть в наличии iPhone Pro Max 15?"
        }
      ],
      "branching": [
        "availability",
        "consultation",
        "help",
        "store_info",
        "reservation",
        "repair",
        "other"
      ]
    },
    "availabilit

In [35]:
if result<0:
    print(f"ошибка {str(answer)}")
else:
    print(f"Полная стоимость анализа: {dictusage.__dict__['total_tokens']} токенов, {cost(dictusage,model)} рублей")
    print(f"{str(answer)}")

Полная стоимость анализа: 7973 токенов, 0.0703 рублей
{'call_center_workflow': {'info': 'info0,1,2,...,n - пояснения ключей', 'root': {'description': 'Входящий звонок клиента. Начало диалога.', 'task': 'Выявление первичной потребности клиента', 'steps': [{'agent_action': 'Добрый день, магазин IT. Меня зовут Игнат. Чем могу помочь?', 'client_action': 'Здравствуйте, меня зовут Артем. Я хотел у вас спросить, а у вас есть в наличии iPhone Pro Max 15?'}], 'branching': ['availability', 'consultation', 'help', 'store_info', 'reservation', 'repair', 'other']}, 'availability': {'description': 'Вопрос клиента по наличию товара', 'task': 'Возможен необязательный диалог перед предоставлением актуальной информации по товару и срокам привоза', 'steps': [{'agent_action': 'Да, уточните, пожалуйста, какой цвет и объем памяти вас интересует?', 'client_action': 'Цвет, в принципе, наверное, не важно. Объем памяти 256 минимальный, по-моему, у него, у Pro Max.'}, {'agent_action': 'Да, все верно, все верно. 

In [ ]:
# копия полного промпта, который надо поделить на отдельные
system=f"""

Ты опытный сотрудник отдела маркетинга и сервиса в компании, которая продает электронику и предоставляет услуги по ремонту и консультации.
Ты специалист по конструктивному ведению диалогов, обладающий высокой компетентностью в вопросах характеристик, особенностей использования и ремонта различных гаджетов.
Ты умеешь вежливо и корректно разговаривать с клиентами, находя подходящие аргументы для убеждения их согласиться на твое предложение.
Ты также умеешь отлично анализировать прошедшие диалоги менеджеров с клиентами, выделять основные тренды, моменты их изменения, замечать сильные стороны и
эффективные приемы менеджера, а также его слабые стороны и ошибки, допущенные в разговоре с клиентом.

Текст диалога между менеджером и клиентом будет предоставлен тебе в формате JSON как список словарей
с репликами менеджера по ключу "1", репликами клиента по ключу "2".

Тебе будет предоставлена в формате JSON блок-схема, описывающая все допустимые сценарии диалога менеджера с клиентом. Схема изначально не содержит текста диалога.
Эта схема служит инструкцией для менеджера о том, как он должен вести диалог с клиентом,
чтобы максимально эффективно привлечь клиента воспользоваться услугами и товарами компании.
Диалог возможен в свободной форме, но содержание и последовательность его фрагментов строго регламентированы.
Блок-схема описывает диалог в виде серии секций и задает разделение на альтернативные ветки по условию и слияние разных веток в случае, если их продолжение идентично.
Секция представлена в формате ключ/словарь;в словаре секции пары ключ/значение описывают допустимое содержание фрагмента диалога и условия ветвлений и слияний.
Фрагмент диалога в данном контексте представляет содержательную часть, в которой менеджер решает определенную подзадачу, которая приближает к достижению главной цели всего диалога.
В общем случае секция может являться только инструкцией перехода и содержать только ключ слияния.
Секция "root" является точкой входа в диалог.
Описание пар ключ/значение для словаря секции:

description: текст - описание ключевого события или условия, определяющего переход к данной секции от предыдущей и означающее начало новой подзадачи для менеджера.

task: текст - основной план или подзадача, которых должен придерживаться менеджер в данном фрагменте диалога для достижения конечной цели всего диалога,
если текст по ключу task пустой, то выполняемая задача не требует обязательного диалога с клиентом.Если task не заполнен, то для содержания секции существенно только условие по ключу description.

steps: список словарей с опциональным одним из ключей agent_action или client_action, предназначенных для реплик менеджера и клиента соответственно;
этот список предназначен для заполнения ChatGPT и изначально пуст.

branching: список ключей, отражающих допустимые варианты ветвления по условию в данной точке диалога; управление переходит только по одному из этих ключей к следующей вложенной секции,
у которой ключ description наиболее близко соответствует условию перехода.

merge: ключ слияния - инструкция перехода, указывающий на секцию, являющуюся одинаковым продолжением нескольких альтернативных веток.
Ключ слияния следует искать на одном уровне с ключом root.

Реальный диалог менеджеру не всегда удается провести в полном соответствии со сценарием, предписанным блок-схемой,
твоя задача установить степень соответствия предоставленного тебе диалога и наиболее подходящего сценария из блок-схемы.

Следуй этим правилам для сопоставления диалога и блок-схемы:
В общем случае развитие диалога конструируется из последовательности секций,
где выбор каждой следующей секции строго ограничен списком по ключу branching или значением по ключу merge, определенными в текущей секции.

1. Сконструируй по блок-схеме все возможные сценарии течения диалога в форме последовательности секций.
2. Для каждого варианта из пункта 1 сформируй отдельный текст-конспект, путем последовательного объединения значений по ключу description и значений по ключу task.
3. Сформируй список пар сценарий/соответствующий сценарию текст-конспект по результатам из пунктов 1 и 2.
4. Используй полученные тексты-конспекты как образцы результатов суммаризации диалогов менеджера и клиента. Суммаризируй переданный тебе диалог менеджера и клиента по образцам.
5. Из всего списка по пункту 3 найди сценарий, для которого соответствующий текст-конспект более всех остальных совпадает с суммаризованным диалогом менеджера и клиента. Используй эмбеддинги для определения степени совпадения.
6. Для последовательности секций по сценарию из пункта 5 раздели полный текст диалога на фрагменты соблюдая условия:
- Каждый фрагмент диалога может содержать одну или несколько реплик менеджера и/или клиента.
- Отдельные фрагменты не должны пересекаться.
- Последовательно объединенные фрагменты диалога, назначенные для финальной серии секций, должны представлять диалог, идентичный исходному. Это означает, что порядок реплик в исходном диалоге должен быть строго сохранён.
- Запрещены потеря, перемешивание реплик или их повторение более чем в одной секции. Реплики должны быть использованы в том же порядке, в котором они встречаются в исходном диалоге.
- интегральное посекционное соответствие фрагментов диалога планам (task) и условиям (description) должно быть максимально возможным.
7. Сохрани как результат финальную серию секций по сценарию из пункта 5 и деление диалога на фрагменты из пункта 6.

Главной целью менеджера в диалоге является привлечение клиента к использованию услуг компании и/или покупке товаров компании.
Твоя цель — оценить прошедший диалог и сделать выводы о его качестве по ряду критериев из словаря evaluation_criteria.
Для оценки по критерию "stopwords":
    - Извлеки из исходного диалога только реплики менеджера, используя ключ "1".
    - Используй ключ "list" для выбора списка запрещённых слов и фраз.
    - Проверь каждую реплику менеджера на наличие запрещённых слов и фраз.
    - Создай список badlist, включив в него только те реплики менеджера, которые содержат запрещённые слова и/или фразы.

**Представление результатов в виде JSON на основе предоставленной тебе блок-схемы:**
1. Создай пустой список с ключом FinalScript на одном уровне с ключом root.
2. Последовательно заполни список FinalScript всеми ключами, по которым строится финальный сценарий.
3. Заполни списки с ключами steps в секциях финального сценария соответствующими фрагментами диалога.
4. Исключи из схемы все альтернативные секции, которые не входят в финальный сценарий.
5. Для каждого критерия в словаре evaluation_criteria оцени диалог по достижению главной цели менеджера и заполни значение по ключу value (чем выше значение, тем выше оценка качества диалога).
6. Для критерия "stopwords" добавь созданный ранее список badlist.
"""

# Транскрибация - получение образца диалога для анализа

In [ ]:
# так можно посмотреть доступные параметры
!whisper --help

In [15]:
# Используем Whisper для обработки аудиофайла
%%time
agentfolder="agent2"
for dirname, _, filenames in os.walk('.'):
    # print(dirname,filenames)
    if agentfolder in dirname:
        os.chdir(agentfolder)
        for fname in filenames:
            if ".mp3" in fname:
                command = f'whisper "{fname}" --model large --language Russian'
                # # Выполняем команду
                get_ipython().system(command)
                # os.rename(f"{dirname}/{fname}", f"{dirname}/done_{fname}")
                print(f"{dirname}/{fname}")


[00:00.460 --> 00:02.880]  Добрый день, магазин АСКС, меня зовут Дмитрий, чем могу помочь?
[00:03.960 --> 00:07.720]  Здравствуйте, я вот по поводу 13-го айфона звонил вам.
[00:08.140 --> 00:09.420]  Угу, розовый.
[00:09.580 --> 00:13.080]  Вот, да, розовый. А он, то есть в наличии, да?
[00:13.600 --> 00:16.000]  Он в наличии, одна штука на Мичурина 120.
[00:18.160 --> 00:21.860]  Так, так, так, так. А забронировать его можно?
[00:22.520 --> 00:25.020]  Да, конечно, могу забронировать. Как вас зовут?
[00:25.160 --> 00:26.800]  То есть, Влад Денис.
[00:27.580 --> 00:27.980]  Угу.
[00:30.000 --> 00:33.480]  То есть, за наличку он сколько будет? 53 900, да?
[00:35.200 --> 00:35.760]  Судутков?
[00:43.860 --> 00:46.160]  Да, 53 900 за наличный расчет.
[00:47.180 --> 00:49.700]  Ага, так, а до скольки работаете там?
[00:50.480 --> 00:51.220]  До 8 вечера.
[00:51.960 --> 00:54.760]  Так, Мичурина 120, это примерно там, ну так?
[00:54.760 --> 00:59.500]  Это, ну, овраг пропольщиков, где Панов

In [6]:
# установка библиотеки pydub
!pip install pydub

In [7]:
from google.colab import userdata
# Получение ключа API от пользователя и установка его как переменной окружения
API_KEY = userdata.get("TINKOFF_API_KEY")
SECRET_KEY = userdata.get("TINKOFF_SECRET")
os.environ["TINKOFF_API_KEY"] = API_KEY
# import getpass
# # передаем API_KEY
# API_KEY = getpass.getpass("Tinkoff API Key:")
# # передаем SECRET_KEY
# SECRET_KEY = getpass.getpass("Tinkoff SECRET Key:")

In [8]:
agentfolder="agent2"
for dirname, _, filenames in os.walk('.'):
    if agentfolder in dirname:
        os.chdir(agentfolder)
        for fname in filenames:
            if ".mp3" in fname:
                print(f"{dirname}/{fname}")


./agent2/2024-05-29_10-02-53.068248_from_79277475995_to_033222_session_3850208577_talk.mp3
./agent2/2024-05-29_10-39-26.101103_from_79279012016_to_033222_session_4024274278_talk.mp3
./agent2/2024-05-29_11-15-20.731390_from_79276078742_to_083378_session_3942956580_talk.mp3
./agent2/2024-05-24_15-28-08.674147_from_79171173909_to_033222_session_4012586103_talk.mp3
./agent2/2024-05-29_14-53-45.513802_from_79276968271_to_033222_session_4039327039_talk.mp3
./agent2/aaf6cfa7-5cdc-4356-a7e9-3c9566e0fb76.mp3


In [9]:
current_directory = os.getcwd()
print(current_directory)

/content/drive/MyDrive/docs4colab/data_structure/agent2


In [16]:
from pydub import AudioSegment
# чтение из файла любого формата
mp3fname="2024-05-24_15-28-08.674147_from_79171173909_to_033222_session_4012586103_talk.mp3"
music = AudioSegment.from_file(file=mp3fname,format='mp3')
# продолжительность аудио (секунды)
print(music.duration_seconds)
# частота дискретизаци
print(music.frame_rate)
# количество каналов
print(music.channels)

377.213875
8000
2


In [17]:
from tinkoff_voicekit_client import ClientSTT
from pprint import pprint

# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",
    "sample_rate_hertz": 8000,
    "num_channels": 2,
    "enable_automatic_punctuation": True  # включаем пунктуацию
    }

# вызываем метод recognize
response = client.recognize(mp3fname,audio_config)

In [18]:
users=['Менеджер','Клиент']
for result in response['results']:
    if result['channel']==1:continue
    print(f"{users[result['channel']]}:",end=" ")
    for alternative in result['alternatives']:
        print(alternative['transcript'])

Менеджер: Добрый день, магазин АСИ, меня зовут Игнат, чем могу помочь?
Менеджер: Здравствуйте, меня зовут Артем.
Менеджер: Я хотел у вас спросить. А у вас есть в наличии?
Менеджер: Айфон.
Менеджер: Промакс пятнадцатый.
Менеджер: Да, уточните, пожалуйста, какой цвет и объем памяти вас интересует?
Менеджер: Цвет, в принципе, наверное, не важно, объем памяти двести пятьдесят шесть минимальный, по-моему, у него промакс. Да, все верно, все верно.
Менеджер: А?
Менеджер: Так, в наличии есть все цвета, черный, синий, натуральный и белый. Актуальная стоимость. Сто девять девятьсот, при оплате наличными.
Менеджер: А сто девять девятьсот, ожидал, он стоит просто, я смотрел на сайте, мне казалось, что у вас там что-то сто двенадцать девятьсот вроде было.
Менеджер: Нет-нет, сто девять, девятьсот, сейчас актуальная цена.
Менеджер: Промакс, Айфа, айфон, пятнадцать бумаг да, получается, да, да, да, все верно, одна физическая, сим-карта плюс еси.
Менеджер: Так скажите, пожалуйста.
Менеджер: А?
Менеджер

In [15]:
users=['Менеджер','Клиент']
for result in response['results']:
    if result['channel']==1:continue
    print(f"{users[result['channel']]}:",end=" ")
    for alternative in result['alternatives']:
        print(alternative['transcript'])

Менеджер: Добрый день, магазин Аскет, меня зовут Игнат, чем могу помочь?
Менеджер: Добрый день, Иван, меня Андрей зовут.
Менеджер: Подскажите, пожалуйста, сейчас секундочку, я вам скажу номер заказа.
Менеджер: Если по вашему номеру телефона я вижу ваш заказ на.
Менеджер: Чехлы пятака.
Менеджер: А что их восемнадцать тысяч, то куда?
Менеджер: А у вас три чехла здесь.
Менеджер: Три чехла.
Менеджер: Да.
Менеджер: А так.
Менеджер: Руслан переместил.
Менеджер: Надо их выбирать. Три часа. А сколько один-то стоит. У вас. Сорок четыре?
Менеджер: Один, шесть, четыреста девяносто.
Менеджер: Тут их два варианта за шесть четыреста девяносто и один вариант за пять девятьсот девяносто.
Менеджер: А?
Менеджер: Это сто пятнадцать промакс.
Менеджер: Да, все верно.
Менеджер: Да хорошо понял. Все теперь понятно. Думаю, что за сумма такая.
Менеджер: Нужно предварить все, понял, спасибо.
Менеджер: До свидания, хорошего дня, до свидания.
Менеджер: 


Добрый день, магазин, а есть есть? Меня зовут Игнатий, могу помочь?
Добрый день, меня Андрей зовут.
Скажите, пожалуйста, секундочку, я вам скажу номер заказа.
Если по вашему номеру телефона, я вижу ваш заказ на чехлы пятака.
А что их 18 тысяч-то куда?
А у вас три чехла здесь.
Три чехла?
Да.
А, это на руках.
Руслан переместил, надо их выбирать.
А сколько один-то стоит у вас в ракетире?
Один 6490, тут их два варианта за 6490 и один вариант за 5 990.
Это что, 15 промакс?
Да, все верно.
Хорошо, все, теперь понятно.
Думаю, что за сумма такая непонятная.
Все, спасибо.
До свидания.
Хорошо, до свидания.
До свидания.